# Advent of Code
## Day 9: Encoding Error
### Part One

With your neighbor happily enjoying their video game, you turn your attention to an open data port on the little screen in the seat in front of you.

Though the port is non-standard, you manage to connect it to your computer through the clever use of several paperclips. Upon connection, the port outputs a series of numbers (your puzzle input).

The data appears to be encrypted with the eXchange-Masking Addition System (XMAS) which, conveniently for you, is an old cypher with an important weakness.

XMAS starts by transmitting a preamble of 25 numbers. After that, each number you receive should be the sum of any two of the 25 immediately previous numbers. The two numbers will have different values, and there might be more than one such pair.

For example, suppose your preamble consists of the numbers `1` through `25` in a random order. To be valid, the next number must be the sum of two of those numbers:

* `26` would be a valid next number, as it could be `1` plus `25` (or many other pairs, like `2` and `24`).
* `49` would be a valid next number, as it is the sum of `24` and `25`.
* `100` would not be valid; no two of the previous 25 numbers sum to `100`.
* `50` would also not be valid; although `25` appears in the previous 25 numbers, the two numbers in the pair must be different.

Suppose the 26<sup>th</sup> number is `45`, and the first number (no longer an option, as it is more than 25 numbers ago) was `20`. Now, for the next number to be valid, there needs to be some pair of numbers among `1`-`19`, `21`-`25`, or `45` that add up to it:

* `26` would still be a valid next number, as `1` and `25` are still within the previous 25 numbers.
* `65` would not be valid, as no two of the available numbers sum to it.
* `64` and 66 would both be valid, as they are the result of `19+45` and `21+45` respectively.

Here is a larger example which only considers the previous 5 numbers (and has a preamble of length 5):

>```
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
>```

In this example, after the 5-number preamble, almost every number is the sum of two of the previous 5 numbers; the only number that does not follow this rule is `127`.

The first step of attacking the weakness in the XMAS data is to find the first number in the list (after the preamble) which is not the sum of two of the 25 numbers before it. What is the first number that does not have this property?

---

In [1]:
# Read in the inputs and convert them to integers
inputs = [int(i[:-1]) for i in open("Day09_input.txt").readlines()]

In [2]:
# deques are fast for adding and removing elements from the ends of
# your list, so let's use them
from collections import deque

# This exception class will help us identify the vulnerabilities in XMAS
class XMASError(Exception):
    pass

# This "Window" class will be built for purpose and help us keep a window of the last n inputs
class Window(deque):
    def advance(self, val):
        """
        advance will append a value to the end of the queue and forget the last element in the queue.
        
        :param val: the value you wish to add to the front of the queue
        :return: self
        """
        # Initialise a truth value
        p = False
        
        # This for loop checks if the new number is the sum of two of the previous numbers in the window
        for i in self:
            p |= (val - i in self)
            # If this condition holds for any two numbers, we can stop checking
            if p:
                break
        
        # If the new number is the sum of two preceding numbers in the window
        if p:
            self.append(val)  # Append the new value
            self.popleft()    # Remove the oldest value
        else:
            # Otherwise, raise an XMASError
            raise XMASError(f"The number {val} is not the sum of two of the preceding {len(self)} numbers.")

In [3]:
# Initialise vars
n = 25  # The preamble is the first 25 numbers
sliding_window = Window(inputs[:n])  # Put these into the new Window class
answer = None  # I want to store the answer, so initialise a global variable

# For each number in the inputs after the preamble
for number in inputs[25:]:
    try:
        # Try to advance the window
        sliding_window.advance(number)
    except XMASError:
        # Unless we get an XMASError, in which case print the solution
        answer = number
        print(f"The first number to not have the desired property is {number:,}.")
        break

The first number to not have the desired property is 10,884,537.


---

### Part Two

The final step in breaking the XMAS encryption relies on the invalid number you just found: you must find a contiguous set of at least two numbers in your list which sum to the invalid number from step 1.

Again consider the above example:

>```
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
>```

In this list, adding up all of the numbers from `15` through `40` produces the invalid number from step 1, `127`. (Of course, the contiguous set of numbers in your actual list might be much longer.)

To find the encryption weakness, add together the smallest and largest number in this contiguous range; in this example, these are `15` and `47`, producing `62`.

What is the encryption weakness in your XMAS-encrypted list of numbers?

---

In [4]:
# Initialise variable to store answer
encryption_weakness = None

# For each number in the inputs
for i, number in enumerate(inputs):
    # And each index after the index i
    for j in range(i, len(inputs)):
        # Check if the sum between index i and index j totals the answer from part one
        if sum(inputs[i:j]) == answer:
            # If it does, print the solution and break the loop
            encryption_weakness = min(inputs[i:j]) + max(inputs[i:j])
            break
        elif sum(inputs[i:j]) >= answer:
            # If it's bigger than the answer, we can stop adding more positive integers on
            break
        else:
            # Otherwise, keep looking!
            pass
    
    # If we've found an answer, we can break out of the outer for loop
    if encryption_weakness is not None:
        break

print(f"The answer is {encryption_weakness:,}.")

The answer is 1,261,309.


---